<a href="https://colab.research.google.com/github/Pathway2008/CarPrice/blob/main/CarPrice_stackRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/CarPrice/train.csv')
test = pd.read_csv('/content/drive/MyDrive/CarPrice/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/CarPrice/sample_submission.csv')

In [ ]:
smallest_values = train['생산년도'].nsmallest(2).index
train = train.drop(smallest_values)

In [ ]:
largest_values = train['주행거리'].nlargest(3).index
train = train.drop(largest_values)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
ordinal_features = ['브랜드', '차량모델명', '판매도시', '판매구역']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 1.9 MB/s eta 0:00:00


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train.drop(['ID', '가격'], axis=1)
y = train['가격']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import pandas as pd
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import optuna
from tqdm import tqdm
from catboost import Pool

In [ ]:
def objective(trial):
    # Define the hyperparameter ranges for CatBoost
    catboost_params = {
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'random_seed': 42,
        'task_type': 'GPU',
        'boosting_type': trial.suggest_categorical('boosting_type', ['Plain', 'Ordered']),
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 1e2),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-5, 1e2),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'use_best_model': True,
        'early_stopping_rounds': 100,
        'verbose': False
    }

    # Define the hyperparameter ranges for XGBoost
    xgboost_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'eta': trial.suggest_loguniform('eta', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-5, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 10.0),
        'seed': 42,
        'verbosity': 0,
        'tree_method': 'gpu_hist'
    }


    # Create the CatBoost model with the updated parameters
    catboost_model = CatBoostRegressor(**catboost_params)

    # Fit the CatBoost model to the training data
    catboost_model.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)

    # Get the best iteration
    best_iteration = catboost_model.best_iteration_

# Create a new CatBoost model with the best iteration specified
    catboost_model_best = CatBoostRegressor(
       **catboost_params,
       iterations=best_iteration,
       use_best_model=True
)

    # Fit the CatBoost model with the best iteration to the training data
    catboost_model_best.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

    # Make predictions on the validation data
    y_pred_valid = catboost_model_best.predict(X_valid)

    xgboost_model = XGBRegressor(**xgboost_params)
    xgboost_model.fit(X_train, y_train)
     # Create the final stacked regression model
    stacked_model = StackingRegressor(
        estimators=[('catboost', catboost_model), ('xgboost', xgboost_model)],
        final_estimator=CatBoostRegressor()
    )

    # Fit the stacked model to the training data
    stacked_model.fit(X_train, y_train)

    # Make predictions on the validation data
    y_pred = stacked_model.predict(X_valid)

    # Calculate the mean absolute error
    mae = mean_absolute_error(y_valid, y_pred)

    return mae

In [ ]:
study = optuna.create_study(direction='minimize')
n_trials = 100
with tqdm(total=n_trials) as pbar:
    def update_pbar(_, __):
        pbar.update(1)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, callbacks=[update_pbar])

Test

In [ ]:
import optuna
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor

In [ ]:
# Split the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def objective(trial):
    # Define the hyperparameter ranges for CatBoost
    catboost_params = {
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'random_seed': 42,
        'task_type': 'GPU',
        'boosting_type': trial.suggest_categorical('boosting_type', ['Plain', 'Ordered']),
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.1, 0.2, 0.3, 0.4, 0.5]),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 1e2),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-5, 1e2),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'verbose': False
    }

    # Define the hyperparameter ranges for XGBoost
    xgboost_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'eta': trial.suggest_categorical('eta', [0.1, 0.2, 0.3, 0.4, 0.5]),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-5, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 10.0),
        'seed': 42,
        'verbosity': 0,
        'tree_method': 'gpu_hist'
    }

    # Create the CatBoost model with the updated parameters
    catboost_model = CatBoostRegressor(**catboost_params)

    # Fit the CatBoost model to the training data
    catboost_model.fit(X_train, y_train)

    # Make predictions on the validation data
    y_pred_valid = catboost_model.predict(X_valid)

    # Create the XGBoost model with the updated parameters
    xgboost_model = XGBRegressor(**xgboost_params)

    # Fit the XGBoost model to the training data
    xgboost_model.fit(X_train, y_train)

    # Create the final stacked regression model
    stacked_model = StackingRegressor(
        estimators=[('catboost', catboost_model), ('xgboost', xgboost_model)],
        final_estimator=CatBoostRegressor(task_type='GPU', verbose=False)
    )

    # Fit the stacked model to the training data
    stacked_model.fit(X_train, y_train)

    # Make predictions on the validation data
    y_pred = stacked_model.predict(X_valid)

    # Calculate the mean absolute error
    mae = mean_absolute_error(y_valid, y_pred)

    return mae

계산안됨 성능부족

In [ ]:
study = optuna.create_study(direction='minimize')
n_trials = 100
with tqdm(total=n_trials) as pbar:
    def update_pbar(_, __):
        pbar.update(1)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, callbacks=[update_pbar])

GridSearch로 변경

In [ ]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

In [ ]:
# Define the parameter grid for CatBoost
catboost_param_grid = {
    'boosting_type': ['Plain', 'Ordered'],
    'iterations': [500, 1000],
    'learning_rate': [ 0.2, 0.5],
    'depth': [4,  8],
    'early_stopping_rounds': [100],
    'task_type': ['GPU'],
}

In [ ]:
# Define the parameter grid for XGBoost
xgboost_param_grid = {
    'booster': ['gbtree', 'dart'],
    'eta': [0.01, 0.05],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 1.0],
    'colsample_bytree': [0.6, 1.0],
    'tree_method': ['gpu_hist'],
}

In [ ]:
# Create the CatBoost model
catboost_model = CatBoostRegressor(loss_function='MAE', eval_metric='MAE', random_seed=42, verbose=False)

# Create the XGBoost model
xgboost_model = XGBRegressor(objective='reg:squarederror', eval_metric='mae', seed=42, verbosity=0)

In [ ]:
stacked_model = StackingRegressor(
    estimators=[('catboost', catboost_model), ('xgboost', xgboost_model)],
    final_estimator=CatBoostRegressor(loss_function='MAE', eval_metric='MAE', random_seed=42, verbose=False)
)

In [ ]:
param_grid = {'catboost__' + k: v for k, v in catboost_param_grid.items()}

In [ ]:
# Perform grid search for finding the optimal parameters
grid_search = GridSearchCV(estimator=stacked_model, param_grid={'catboost__' + k: v for k, v in catboost_param_grid.items()}, scoring='neg_mean_absolute_error')

In [ ]:
study = optuna.create_study(direction='minimize')
n_trials = 100
with tqdm(total=n_trials) as pbar:
    def update_pbar(_, __):
        pbar.update(1)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, callbacks=[update_pbar])

In [ ]:
total_combinations = len(grid_search.param_grid)
with tqdm(total=total_combinations, desc="Grid Search", ncols=80) as pbar:
    def update_pbar(*args, **kwargs):
        pbar.update(1)

    grid_search.fit(X_train, y_train)

In [ ]:
# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

In [ ]:
best_estimator.fit(X_train, y_train)

In [ ]:
y_pred = best_estimator.predict(X_valid)

stacked regression으로 optimal parameter를 찾는건 불가능
optimal을 찾고 stacked regression을 만들어야함